<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Interview-with-Person-X" data-toc-modified-id="Interview-with-Person-X-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interview with Person X</a></span></li><li><span><a href="#Panels" data-toc-modified-id="Panels-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Panels</a></span></li></ul></div>

In [ ]:
# Imports. Run this first!

from query.models import LabeledInterview, LabeledPanel, LabeledCommercial, Video, FaceIdentity
from esper.rekall import *
from rekall.temporal_predicates import *
from rekall.spatial_predicates import *
from rekall.bbox_predicates import *
from rekall.payload_predicates import *
from rekall.parsers import *
from rekall.merge_ops import *
from rekall.logical_predicates import *
from rekall.video_interval_collection import VideoIntervalCollection
from esper.prelude import esper_widget
#from esper.captions import topic_search
from django.db.models import FloatField

sandbox_videos = [529, 763, 2648, 3459, 3730, 3769, 3952, 4143, 4611, 5281, 6185, 7262, 8220,
    8697, 8859, 9215, 9480, 9499, 9901, 10323, 10335, 11003, 11555, 11579, 11792,
    12837, 13058, 13141, 13247, 13556, 13827, 13927, 13993, 14482, 15916, 16215,
    16542, 16693, 16879, 17458, 17983, 19882, 19959, 20380, 20450, 23181, 23184,
    24193, 24847, 24992, 25463, 26386, 27188, 27410, 29001, 31378, 32472, 32996,
    33004, 33387, 33541, 33800, 34359, 34642, 36755, 37107, 37113, 37170, 38275,
    38420, 40203, 40856, 41480, 41725, 42756, 45472, 45645, 45655, 45698, 48140,
    49225, 49931, 50164, 50561, 51175, 52075, 52749, 52945, 53355, 53684, 54377,
    55711, 57384, 57592, 57708, 57804, 57990, 59122, 59398, 60186]

# Interview with Person X

In [ ]:
guest_name = "bernie sanders"

In [ ]:
# Load the data

identities = FaceIdentity.objects.filter(face__shot__video_id__in=sandbox_videos)
hosts_qs = identities.filter(face__is_host=True)
guest_qs = identities.filter(identity__name=guest_name).filter(probability__gt=0.7)

# Note that our unit of time is frames
hosts = VideoIntervalCollection.from_django_qs(
    hosts.annotate(video_id=F("face__shot__video_id"),
                  min_frame=F("face__shot__min_frame"),
                  max_frame=F("face__shot__max_frame"))
)
guest = VideoIntervalCollection.from_django_qs(
    sanders.annotate(video_id=F("face__shot__video_id"),
                  min_frame=F("face__shot__min_frame"),
                  max_frame=F("face__shot__max_frame"))
)

In [ ]:
# Interview query

# Get all shots where the guest and a host are on screen together
guest_with_host = guest.overlaps(hosts).coalesce()

# This temporal predicate defines A overlaps with B, or A before by less than 10 frames,
#   or A after B by less than 10 frames
overlaps_before_or_after_pred = or_pred(
    or_pred(overlaps(), before(max_dist=10), arity=2),
    after(max_dist=10), arity=2)

# This code finds sequences of:
#   guest with host overlaps/before/after host OR
#   guest with host overlaps/before/after guest
interview_candidates = guest_with_host \
    .merge(hosts, predicate=overlaps_before_or_after_pred) \
    .set_union(guest_with_host.merge(
        guest, predicate=overlaps_before_or_after_pred)) \
    .coalesce()

# Sequences may be interrupted by shots where the guest or host don't
#   appear, so dilate and coalesce to merge neighboring segments
interviews = interview_candidates \
    .dilate(600) \
    .coalesce() \
    .dilate(-600) \
    .filter_length(min_length=1350)

In [ ]:
# Display the results in the Esper widget
esper_widget(intrvllists_to_result(interviews.get_allintervals()))

In [ ]:
# Compare against ground truth (only available for Bernie Sanders, John McCain, Kellyanne Conway)
ground_truth_interviews = LabeledInterview.objects.annotate(fps=F('video__fps')).annotate(
    min_frame=F('fps') * F('start'),
    max_frame=F('fps') * F('end')
).filter(guest1=guest_name)

result = intrvllists_to_result(
    VideoIntervalCollection.from_django_qs(ground_truth_interviews).get_allintervals(),
    color='black')
add_intrvllists_to_result(result, interviews.get_allintervals(), color='red')

esper_widget(result)

# Panels

In [ ]:
# Load Data
faces_qs = Face.objects.filter(shot__video_id__in=sandbox_videos).annotate(
    video_id=F("shot__video_id"),
    min_frame=F("shot__min_frame"),
    max_frame=F("shot__max_frame")
)

# One interval for each face
faces = VideoIntervalCollection.from_django_qs(
    faces_qs,
    with_payload=in_array(bbox_payload_parser(VideoIntervalCollection.django_accessor)))

# Merge shots
faces = faces.coalesce(payload_merge_op=payload_plus)

In [ ]:
three_faces_scene_graph = {
    'nodes': [
        { 'name': 'face1', 'predicates': [height_at_least(0.25)] },
        { 'name': 'face2', 'predicates': [height_at_least(0.25)] },
        { 'name': 'face3', 'predicates': [height_at_least(0.25)] },
    ],
    'edges': [
        { 'start': 'face1', 'end': 'face2', 'predicates': [same_value('y1', epsilon=0.05)] },
        { 'start': 'face2', 'end': 'face3', 'predicates': [same_value('y1', epsilon=0.05)] }
    ]
}
panels = faces.filter(payload_satisfies(scene_graph(three_faces_scene_graph, exact=True)))

In [ ]:
esper_widget(intrvllists_to_result_bbox(panels.get_allintervals()))